# Advent of Code

## 2022-012-017
## 2022 017

https://adventofcode.com/2022/day/17

In [3]:
import time

# Define the rock shapes
ROCK_SHAPES = [
    [(0, 0), (1, 0), (2, 0), (3, 0)],  # Horizontal line
    [(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)],  # Plus shape
    [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],  # Reverse L shape
    [(0, 0), (0, 1), (0, 2), (0, 3)],  # Vertical line
    [(0, 0), (1, 0), (0, 1), (1, 1)],  # Square
]

# Function to parse the input
def parse_input(file_path):
    with open(file_path, 'r') as f:
        return f.read().strip()

# Function to check for collisions
def is_collision(rock, position, stopped_rocks, width=7):
    for dx, dy in rock:
        x, y = position[0] + dx, position[1] + dy
        if x < 0 or x >= width or y < 0 or (x, y) in stopped_rocks:
            return True
    return False

# Function to simulate falling rocks
def simulate_rocks(jet_pattern, rock_shapes, num_rocks, chamber_width=7):
    stopped_rocks = set()
    max_height = 0
    jet_index = 0

    for rock_index in range(num_rocks):
        rock = rock_shapes[rock_index % len(rock_shapes)]
        position = (2, max_height + 3)  # Start position

        while True:
            # Apply jet push
            jet = jet_pattern[jet_index % len(jet_pattern)]
            jet_index += 1
            new_position = (position[0] + (1 if jet == '>' else -1), position[1])
            if not is_collision(rock, new_position, stopped_rocks, chamber_width):
                position = new_position

            # Apply gravity
            new_position = (position[0], position[1] - 1)
            if is_collision(rock, new_position, stopped_rocks, chamber_width):
                # Rock comes to rest
                for dx, dy in rock:
                    stopped_rocks.add((position[0] + dx, position[1] + dy))
                    max_height = max(max_height, position[1] + dy + 1)
                break
            else:
                position = new_position

    return max_height

# Start timer
start_time = time.perf_counter()

# Parse input and simulate
file_path = 'input.txt'
jet_pattern = parse_input(file_path)
tower_height = simulate_rocks(jet_pattern, ROCK_SHAPES, num_rocks=2022)

# End timer
end_time = time.perf_counter()

# Output result and time taken
print(f"Tower Height after 2022 rocks: {tower_height}")
print(f"Elapsed Time: {end_time - start_time:.9f} s")

Tower Height after 2022 rocks: 3085
Elapsed Time: 0.081593300 s


In [4]:
def simulate_optimized(jet_pattern, rock_shapes, num_rocks, chamber_width=7):
    stopped_rocks = set()
    max_height = 0
    jet_index = 0

    state_map = {}  # To detect cycles: {(jet_index, rock_index, top_shape): (rock_count, height)}
    cycle_found = False
    cycle_height = 0
    cycle_rock_count = 0

    rock_count = 0
    while rock_count < num_rocks:
        rock_index = rock_count % len(rock_shapes)
        rock = rock_shapes[rock_index]
        position = (2, max_height + 3)  # Start position

        while True:
            # Apply jet push
            jet = jet_pattern[jet_index % len(jet_pattern)]
            jet_index += 1
            new_position = (position[0] + (1 if jet == '>' else -1), position[1])
            if not is_collision(rock, new_position, stopped_rocks, chamber_width):
                position = new_position

            # Apply gravity
            new_position = (position[0], position[1] - 1)
            if is_collision(rock, new_position, stopped_rocks, chamber_width):
                # Rock comes to rest
                for dx, dy in rock:
                    stopped_rocks.add((position[0] + dx, position[1] + dy))
                    max_height = max(max_height, position[1] + dy + 1)
                break
            else:
                position = new_position

        # Update rock count
        rock_count += 1

        # Check for cycle detection
        if not cycle_found:
            # Record the top shape of the tower to detect cycles
            top_shape = tuple((x, max_height - y) for x, y in stopped_rocks if max_height - y <= 20)
            state = (jet_index % len(jet_pattern), rock_index, top_shape)
            if state in state_map:
                # Cycle detected
                prev_rock_count, prev_height = state_map[state]
                cycle_rock_count = rock_count - prev_rock_count
                cycle_height = max_height - prev_height
                cycle_found = True
                remaining_rocks = num_rocks - rock_count
                full_cycles = remaining_rocks // cycle_rock_count
                max_height += full_cycles * cycle_height
                rock_count += full_cycles * cycle_rock_count
            else:
                state_map[state] = (rock_count, max_height)

    return max_height


# Start timer
start_time = time.perf_counter()

# Parse input and simulate for 1 trillion rocks
tower_height_large = simulate_optimized(jet_pattern, ROCK_SHAPES, num_rocks=1000000000000)

# End timer
end_time = time.perf_counter()

# Output result and time taken
print(f"Tower Height after 1000000000000 rocks: {tower_height_large}")
print(f"Elapsed Time: {end_time - start_time:.9f} s")

KeyboardInterrupt: 

In [5]:
def simulate_optimized(jet_pattern, rock_shapes, num_rocks, chamber_width=7):
    stopped_rocks = set()  # To store the coordinates of settled rocks
    max_height = 0
    jet_index = 0

    state_map = {}  # For cycle detection: {(jet_index, rock_index, top_shape): (rock_count, height)}
    cycle_found = False

    rock_count = 0
    while rock_count < num_rocks:
        rock_index = rock_count % len(rock_shapes)
        rock = rock_shapes[rock_index]
        position = (2, max_height + 3)  # Start position

        while True:
            # Apply jet push
            jet = jet_pattern[jet_index % len(jet_pattern)]
            jet_index += 1
            new_position = (position[0] + (1 if jet == '>' else -1), position[1])
            if not is_collision(rock, new_position, stopped_rocks, chamber_width):
                position = new_position

            # Apply gravity
            new_position = (position[0], position[1] - 1)
            if is_collision(rock, new_position, stopped_rocks, chamber_width):
                # Rock comes to rest
                for dx, dy in rock:
                    stopped_rocks.add((position[0] + dx, position[1] + dy))
                    max_height = max(max_height, position[1] + dy + 1)
                break
            else:
                position = new_position

        # Update rock count
        rock_count += 1

        # Check for cycle detection
        if not cycle_found:
            # Only keep the top 20 rows of the tower for state
            top_shape = tuple((x, max_height - y) for x, y in stopped_rocks if max_height - y <= 20)
            state = (jet_index % len(jet_pattern), rock_index, top_shape)
            if state in state_map:
                # Cycle detected
                prev_rock_count, prev_height = state_map[state]
                cycle_rock_count = rock_count - prev_rock_count
                cycle_height = max_height - prev_height
                remaining_rocks = num_rocks - rock_count

                # Skip remaining cycles
                full_cycles = remaining_rocks // cycle_rock_count
                max_height += full_cycles * cycle_height
                rock_count += full_cycles * cycle_rock_count
                cycle_found = True
            else:
                state_map[state] = (rock_count, max_height)

    return max_height

In [7]:
def simulate_optimized(jet_pattern, rock_shapes, num_rocks, chamber_width=7):
    recent_rows = [{} for _ in range(20)]  # Store the last 20 rows
    base_height = 0  # Tracks the height of rows
    max_height = 0  # Maximum tower height
    jet_pattern_cycle = [1 if jet == '>' else -1 for jet in jet_pattern]
    jet_index = 0
    state_map = {}
    cycle_found = False

    rock_count = 0
    while rock_count < num_rocks:
        rock_index = rock_count % len(rock_shapes)
        rock = rock_shapes[rock_index]
        position = (2, max_height + 3)  # Start position

        while True:
            # Apply jet push
            jet_push = jet_pattern_cycle[jet_index % len(jet_pattern_cycle)]
            jet_index += 1
            new_position = (position[0] + jet_push, position[1])
            if not is_collision(rock, new_position, recent_rows, base_height, chamber_width):
                position = new_position

            # Apply gravity
            new_position = (position[0], position[1] - 1)
            if is_collision(rock, new_position, recent_rows, base_height, chamber_width):
                # Rock comes to rest
                for dx, dy in rock:
                    x, y = position[0] + dx, position[1] + dy
                    if y - base_height >= len(recent_rows):
                        # Extend recent_rows
                        recent_rows.append(set())
                    recent_rows[y - base_height].add(x)
                    max_height = max(max_height, y + 1)
                break
            else:
                position = new_position

        # Update rock count
        rock_count += 1

        # Cycle detection
        if not cycle_found:
            top_shape = tuple(sorted((x, max_height - base_height - y) for y, row in enumerate(recent_rows[-20:]) for x in row))
            state = (jet_index % len(jet_pattern_cycle), rock_index, top_shape)
            if state in state_map:
                prev_rock_count, prev_height = state_map[state]
                cycle_rock_count = rock_count - prev_rock_count
                cycle_height = max_height - prev_height
                remaining_rocks = num_rocks - rock_count

                full_cycles = remaining_rocks // cycle_rock_count
                max_height += full_cycles * cycle_height
                rock_count += full_cycles * cycle_rock_count
                cycle_found = True
            else:
                state_map[state] = (rock_count, max_height)

    return max_height


In [6]:
# Parse input and simulate for 1 trillion rocks
tower_height_large = simulate_optimized(jet_pattern, ROCK_SHAPES, num_rocks=1000000000000)
print(f"Tower Height after 1000000000000 rocks: {tower_height_large}")

KeyboardInterrupt: 

In [12]:
tower_height = simulate_optimized(jet_pattern, ROCK_SHAPES, 1000000000000)

# Print the result
print(f"Tower Height after {num_rocks} rocks: {tower_height}")

AttributeError: 'dict' object has no attribute 'add'

In [11]:
def is_collision(rock, position, recent_rows, base_height, width=7):
    px, py = position
    for dx, dy in rock:
        x, y = px + dx, py + dy
        row_index = y - base_height
        if x < 0 or x >= width or y < 0 or (0 <= row_index < len(recent_rows) and x in recent_rows[row_index]):
            return True
    return False
